In [1]:
from functions_pt import *
import pickle
from torch.optim import Adam

In [2]:
batch_size = 64
lr = 1e-4
clip = 1
epochs = 10

In [3]:
# load data
ys = np.load('ys.npy')
weights = np.load('weights.npy')
length = np.load('length.npy')
words_embed = np.load('possible_words_embed.npy')
with open('indexes.pkl', 'rb') as f:
    indexes = pickle.load( f)

# train/val split
np.random.seed(0)
train_idx = np.random.rand(ys.shape[0])>0.2
val_idx = np.logical_not(train_idx)

ys_train, ys_val = ys[train_idx], ys[val_idx]
weights_train, weights_val = weights[train_idx], weights[val_idx]
length_train, length_val = length[train_idx], length[val_idx]
indexes_train = [idx for idx,bol in zip(indexes,train_idx) if bol]
indexes_val = [idx for idx,bol in zip(indexes,train_idx) if not bol]

# set up dataloader
dataset_train = CustomDataset(ys_train,length_train,indexes_train,words_embed)
sampler = WeightedRandomSampler(weights_train, 50000)
dataset_train = DataLoader(dataset_train, batch_size=batch_size, sampler=sampler,collate_fn=collate)

dataset_val = CustomDataset(ys_val,length_val,indexes_val,words_embed)
sampler = WeightedRandomSampler(weights_val, 10000)
dataset_val = DataLoader(dataset_val, batch_size=batch_size, sampler=sampler,collate_fn=collate)

In [4]:
for _ in range(25):
    embed_size = np.random.choice([64,128,160])
    dropout = np.random.rand()*0.35
    multiple_factor = np.random.choice([1,2])
    layers = np.random.choice([2,3,4])
    agg = np.random.choice(['mean','sum','max']) # mean,sum,min,max
    model = pointNet(layers,embed_size,agg,dropout,multiple_factor).to('cuda')
    opt = Adam([    {'params': [p for p in model.parameters() if p is not model.w]},\
                    {'params': model.w, 'lr': lr/4}
                ],lr=lr)
    _,val_loss = train(opt,model,epochs,dataset_train,dataset_val,model.parameters(),clip,verbose=False,save=False)
    print('size:{}, mul_factor:{}, layers:{}, agg:{}, dropout:{}, MSE:{}\n'.
          format(embed_size,multiple_factor,layers,agg,dropout,val_loss))

size:64, mul_factor:1, layers:4, agg:max, dropout:0.32450477542810136, MSE:21.977670673280954

size:160, mul_factor:2, layers:4, agg:sum, dropout:0.00825110002164419, MSE:3.170150564983487

size:64, mul_factor:2, layers:3, agg:sum, dropout:0.28283533250553783, MSE:3.8977135643363

size:128, mul_factor:2, layers:4, agg:mean, dropout:0.30103819777023416, MSE:9.27444832213223

size:160, mul_factor:2, layers:4, agg:sum, dropout:0.08712807594237947, MSE:3.6702809017151594

size:128, mul_factor:1, layers:3, agg:max, dropout:0.29262215212133097, MSE:12.708432290703058



KeyboardInterrupt: 

In [9]:
torch.save(model.state_dict(), 'baseline0.pt')

In [6]:
w = dataset_val.dataset.words_embed
l = torch.tensor([2309.0])

In [7]:
model((torch.tensor(w).to('cuda'),torch.tensor(l).to('cuda')))

/tmp/ipykernel_40386/3274103529.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model((torch.tensor(w).to('cuda'),torch.tensor(l).to('cuda')))


tensor([2.8766], device='cuda:0', grad_fn=<AddBackward0>)